In [30]:
from cellcalc import get_primitive_hkl, MID, get_pri_vec_inplane
from interface_generator import core, print_near_axis, convert_vector_index
from csl_generator import print_list, getsigmas, get_theta_m_n_list
from numpy import array
from numpy import array, dot, round
from numpy.linalg import inv, norm
import numpy as np
import glob
import shutil
import os
from cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, MID
from interface_generator import core, print_near_axis, convert_vector_index, write_trans_file
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv

import numpy as np
import pandas as pd
import shutil
import os
import math
import glob
from scipy.spatial import distance
import matplotlib.pyplot as plt
from numpy.linalg import inv, norm
from numpy import dot

from Element import Element

In [31]:
my_interface = core('Cu_mp-30_conventional_standard.cif','Cu_mp-30_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [32]:
from interface_generator import super_cell

In [33]:
def get_a_b(CSL, axis):
    hkl_perp_axis = MID(CSL, axis)
    a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    return a.T, b.T

In [34]:
def get_STGB_MLs(CSL, n_1, n_2):
    hkl_1 = MID(CSL, n_1)
    hkl_2 = MID(CSL, n_2)

    return hkl_1, hkl_2

In [ ]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, axis)
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [125]:
def get_expansion_xyz(cell):
    exp_x = ceil(100/norm(cell[:,0]))
    exp_y = ceil(20/norm(cell[:,1]))
    exp_z = ceil(20/norm(cell[:,2]))
    return exp_x, exp_y, exp_z

In [36]:
def define_bicrystal_regions(xhi):
    tol = 1e-5
    """
    generate a file defining some regions in the LAMMPS and define the atoms
    inside these regions into some groups.
    argument:
    region_names --- list of name of regions
    region_los --- list of the low bounds
    region_his --- list of the hi bounds
    """
    end_fixbulk1 = xhi/2-30
    start_fixbulk2 = xhi/2+30
    start_middle = xhi/2-20
    end_middle = xhi/2+20
    start_right = xhi/2 + tol
    start_bulk = 160
    end_bulk = 165


    with open('blockfile', 'w') as fb:
        fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(end_fixbulk1))
        fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_fixbulk2))
        fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_middle,end_middle))
        fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_right))
        fb.write('region left block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_right))
        fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_bulk,end_bulk))
        fb.write('group fixbulk1 region fixbulk1 \n')
        fb.write('group fixbulk2 region fixbulk2 \n')
        fb.write('group middle region middle \n')
        fb.write('group right region right \n')
        fb.write("group left region left \n")
        fb.write('group bulk region bulk \n')

In [37]:
def define_bicrystal_regions2(xhi):
    half_cell = xhi/2
    text = f"""

displace_atoms right move ${{tx}} ${{ty}} ${{tz}} units box

group left_dash region left 

group deleted_atoms subtract left_dash left

displace_atoms deleted_atoms move {half_cell} 0 0 units box

group fixbulk1_dash region fixbulk1 
group fixbulk2_dash region fixbulk2 
group middle_dash region middle 
group bulk_dash region bulk
            """
    with open("blockfile2", "w") as f:
        f.writelines(text)

In [127]:

max_sigma = 100
eps = 1e-5

element = Element("Cu")
cif_filename = "Cu_mp-30_conventional_standard.cif"
my_interface = core(cif_filename, cif_filename)
my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
factor = element.getLatP()/(norm(my_interface.conv_lattice_1[:,0]))
my_interface.lattice_1 =  my_interface.lattice_1*factor
my_interface.lattice_2 =  my_interface.lattice_2*factor
my_interface.conv_lattice_1 =  my_interface.conv_lattice_1*factor
my_interface.conv_lattice_2 =  my_interface.conv_lattice_2*factor  





Warning!, this programme will rewrite the POSCAR file in this dir!


In [128]:
axis = np.array([1,1,1])

# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis, np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis,a,b))
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,my_interface.lattice_1,my_interface.atoms_1,my_interface.elements_1)
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [129]:
def get_thetas(axis,max_sigma):
    theta_list = []
    sigma_list = []
    lists, thetas = getsigmas(axis, max_sigma)
    return thetas

In [92]:
getsigmas([1,1,1], max_sigma)

([3, 7, 13, 19, 31, 37, 43, 49, 61, 67, 73, 79, 91, 97],
 [59.99999999999999,
  38.21321070173819,
  27.79577249602797,
  46.82644889274108,
  17.896551129254163,
  50.569992092103575,
  15.17817893794989,
  43.573578596523625,
  52.659006983369785,
  24.432697679454535,
  11.635051288887157,
  33.99217611435437,
  10.41743820571022,
  30.590688602809458])

In [120]:
os.getcwd()

'C:\\Users\\hatay\\calc\\220418\\test_program'

In [119]:
os.chdir("..")

In [130]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
# my_interface.search_one_position(rotation_axis,38.21,1,0.01)

axis_list = axis.tolist()

thetas = get_thetas(axis_list,max_sigma)


for theta in thetas:
    my_interface.search_one_position(rotation_axis,theta-0.001,1,0.001)

    #     my_interface.search_one_position(axis,theta_list[i][j]-0.001,1,0.001)
    # CSL = my_interface.CSL
    hkl_perp_axis = MID(my_interface.CSL, axis)
    # a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T

    a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    hkl_1 = MID(my_interface.lattice_1,a.T)
    hkl_2 = MID(my_interface.lattice_1,b.T)

    # hkl_1, hkl_2 = get_STGB_MLs(my_interface.lattice_1, n_1, n_2)

    # hkl_name_1 = get_primitive_hkl(hkl_1, my_interface.lattice_1, my_interface.conv_lattice_1)
    # hkl_name_2 = get_primitive_hkl(hkl_2, my_interface.lattice_2, my_interface.conv_lattice_2)

    get_gb_files(my_interface,axis.tolist(),hkl_1,element)


    get_gb_files(my_interface,axis.tolist(),hkl_2,element)

[3.62 3.62 3.62]
rotation matrix = [[ 0.66667674 -0.33333333  0.66665659]
 [ 0.66665659  0.66667674 -0.33333333]
 [-0.33333333  0.66665659  0.66667674]]
Congrates, we found an appx CSL!

U1 = 
[[ 0  0  1]
 [ 1  1  0]
 [ 0 -1  0]]; sigma_1 = 1

U2 = 
[[0 0 1]
 [0 1 0]
 [1 0 0]]; sigma_2 = 1

D = 
[[ 1.000e+00 -1.008e-05  1.008e-05]
 [ 1.008e-05  1.000e+00 -1.008e-05]
 [-1.008e-05  1.008e-05  1.000e+00]]

axis = [3.62 3.62 3.62] ; theta = 59.999

cell 1:
[[ 0  1  0]
 [-1  0  1]
 [ 0  0 -2]]
cell 2:
[[ 0  1  0]
 [-1  0  1]
 [ 0  0 -2]]
x, y, z = 20.0, 4.0, 3.0
[[ 0  1  0]
 [-1  0  1]
 [ 0  0 -2]]
[[ 0  1  0]
 [ 0  0  2]
 [-1  0 -1]]
num of atoms1 = 24
[[ 1.02389062e+02 -1.08916015e-15  4.44577745e-15]
 [ 6.49579434e-16  2.50800957e+01  8.62940323e-16]
 [ 2.54827121e-15  1.21945394e-15  2.66014586e+01]]
cell 1:
[[ 0  0  1]
 [ 1  1  0]
 [ 1 -1  0]]
cell 2:
[[ 0  0  1]
 [ 1  1  0]
 [ 1 -1  0]]
x, y, z = 12.0, 4.0, 4.0
[[ 0  0  1]
 [ 1  1  0]
 [ 1 -1  0]]
[[ 0  0  1]
 [-1  1  0]
 [ 2  0  0]]


[[ 1.16291619e+02  6.73679813e-15 -1.33975695e-15]
 [-6.27646880e-16  2.50800957e+01 -7.25599873e-16]
 [-8.68579796e-16  2.11927424e-15  3.35704989e+01]]
[3.62 3.62 3.62]
rotation matrix = [[ 0.81633455 -0.30611916  0.48978461]
 [ 0.48978461  0.81633455 -0.30611916]
 [-0.30611916  0.48978461  0.81633455]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0 -8 -3]
 [ 0  3 -5]]; sigma_1 = 49

U2 = 
[[-1  0  0]
 [ 0 -5  3]
 [ 0 -3 -8]]; sigma_2 = 49

D = 
[[ 1.000e+00 -1.008e-05  1.008e-05]
 [ 1.008e-05  1.000e+00 -1.008e-05]
 [-1.008e-05  1.008e-05  1.000e+00]]

axis = [3.62 3.62 3.62] ; theta = 43.5725786

cell 1:
[[ 0 -1  0]
 [-8  0 -2]
 [ 3  0 13]]
cell 2:
[[ 0 -1  0]
 [-8  0 -2]
 [ 3  0 13]]
x, y, z = 3.0, 4.0, 1.0
[[ 0 -1  0]
 [-8  0 -2]
 [ 3  0 13]]
[[  0  -1   0]
 [ -5   0 -11]
 [ -3   0  13]]
num of atoms1 = 1176
[[ 1.07508515e+02  3.53386229e-15 -7.47062342e-15]
 [-1.18662507e-15  2.50800957e+01  9.80113064e-16]
 [-3.72575761e-15 -1.43151347e-15  6.20700700e+01]]
cell 1:
[[

[[ 1.00841531e+02 -4.75720717e-15 -2.17516741e-15]
 [ 5.60338404e-15  2.50800957e+01 -2.47641247e-15]
 [-8.19558093e-15  1.30339143e-15  8.73313276e+01]]
cell 1:
[[  0  -1   0]
 [-14   0  -8]
 [ -5   0  11]]
cell 2:
[[  0  -1   0]
 [-14   0  -8]
 [ -5   0  11]]
x, y, z = 2.0, 4.0, 1.0
[[  0  -1   0]
 [-14   0  -8]
 [ -5   0  11]]
[[  0  -1   0]
 [ -5   0 -11]
 [-14   0   8]]
num of atoms1 = 2328
[[ 1.74662655e+02 -3.50562262e-15  2.25187496e-15]
 [ 5.34268614e-15  2.50800957e+01 -8.33636766e-15]
 [ 7.99214289e-15 -9.37962745e-16  5.04207655e+01]]


In [126]:
def get_gb_files(interface,axis,hkl,element):
    interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-5)
    half_cell = dot(interface.lattice_1, interface.bicrystal_U1)
    x,y,z = get_expansion_xyz(half_cell)
    print(f"x, y, z = {x}, {y}, {z}")
    xhi = 2*x*norm(half_cell[:,0])
    
    hkl_name = get_primitive_hkl(hkl, my_interface.lattice_1, my_interface.conv_lattice_1)
    hkl_name = np.array(hkl_name, dtype=int)
    hkl_name_num = hkl_name.copy()
    hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
    hkl_x, hkl_y, hkl_z = hkl_name
    dirname = f"{str(hkl_x)}_{str(hkl_y)}_{str(hkl_z)}"
    os.makedirs(dirname,exist_ok=True)
    os.chdir(dirname) 
    
    if (axis == [1,1,1]):    
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = True)
    else:
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)

    define_bicrystal_regions(xhi)
    define_bicrystal_regions2(xhi)
    
    with open(f"{element.element}.meam","w") as f:
        f.writelines(element.meam())

    with open("library.meam","w") as f:
        f.writelines(element.library_meam())

    with open("proto.in","w") as f:
        f.writelines(element.proto())


    os.chdir(os.pardir)

In [7]:
axis = np.array([1,1,1])

In [8]:
print_list(axis,100)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  60.00 
Sigma:     7  Theta:  38.21 
Sigma:    13  Theta:  27.80 
Sigma:    19  Theta:  46.83 
Sigma:    21  Theta:  21.79 
Sigma:    31  Theta:  17.90 
Sigma:    37  Theta:  50.57 
Sigma:    39  Theta:  32.20 
Sigma:    43  Theta:  15.18 
Sigma:    49  Theta:  43.57 
Sigma:    57  Theta:  13.17 
Sigma:    61  Theta:  52.66 
Sigma:    67  Theta:  24.43 
Sigma:    73  Theta:  11.64 
Sigma:    79  Theta:  33.99 
Sigma:    91  Theta:  10.42 
Sigma:    93  Theta:  42.10 
Sigma:    97  Theta:  30.59 


In [9]:
# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis, np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis,a,b))
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,my_interface.lattice_1,my_interface.atoms_1,my_interface.elements_1)
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [10]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]
my_interface.parse_limit(du = 1e-4, S  = 1e-4, sgm1=100, sgm2=100, dd = 1e-4)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(rotation_axis,38.21,1,0.01)

[3.621262 3.621262 3.621262]
rotation matrix = [[ 0.85716597 -0.28570042  0.42853445]
 [ 0.42853445  0.85716597 -0.28570042]
 [-0.28570042  0.42853445  0.85716597]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0  3  1]
 [ 0 -1  2]]; sigma_1 = 7

U2 = 
[[-1  0  0]
 [ 0  2 -1]
 [ 0  1  3]]; sigma_2 = 7

D = 
[[ 1.000e+00 -3.235e-05  3.235e-05]
 [ 3.235e-05  1.000e+00 -3.235e-05]
 [-3.235e-05  3.235e-05  1.000e+00]]

axis = [3.621262 3.621262 3.621262] ; theta = 38.21



In [11]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, axis)
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [12]:
my_interface.compute_bicrystal(hkl_1, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-2)

cell 1:
[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]
cell 2:
[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]


In [13]:
my_interface.get_bicrystal(xyz_1 = [2,1,1], xyz_2 = [2,1,1],filetype="LAMMPS",filename="atominfile")

[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]
[[ 0 -1  0]
 [ 2  0  4]
 [ 1  0 -5]]
num of atoms1 = 168
num of atoms2 = 168
[[ 2.70990434e+01  6.01710864e-16  6.91810869e-16]
 [-1.95023943e-15  6.27220977e+00 -4.40441118e-15]
 [-3.40457860e-16  5.29203675e-16  2.34684600e+01]]
